<img src="../../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Population Coding Exercises</h1> 
<h2 align="center">Summer Workshop on the Dynamic Brain</h2> 

In [ ]:
import multiprocessing as mp
import os
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import (KFold, LeaveOneOut, RepeatedKFold,
                                     RepeatedStratifiedKFold, StratifiedKFold)
from tqdm import tqdm

import brain_observatory_utilities.datasets.behavior.data_formatting as behavior_utils
from allensdk.brain_observatory.behavior.behavior_project_cache.\
    behavior_neuropixels_project_cache \
    import VisualBehaviorNeuropixelsProjectCache
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [ ]:
import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = "/Volumes/Brain2024/"
    mp.set_start_method('fork')
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/"

In [ ]:
def make_response_array(spike_times, stimulus_presentations, units, window=.05):

    '''
    Create an array of spike counts x stimulus presentations, and a corresponding list of stimulus labels.
    spike_times: spike times 
    stimulus_presentation: stimulus presentation table
    units: units table containing only the units to get the responses of
    '''

    # Sort spike times chronologically; necessary for the binary search later
    sorted_spikes = dict()
    for iu in units.index:
        # Use mergesort/timsort since most spike_times are already sorted
        sorted_spikes[iu] = np.sort(spike_times[iu], kind='mergesort')

    # Create our own copy of stimulus presentations and sort by presentation start time chronologically
    # Sorting of stimulus_presentations isn't necessary, but it speeds up the vectorized `searchsorted(...)`
    stimulus_presentations = stimulus_presentations.sort_values(by='start_time', kind='mergesort', inplace=False)

    # Calculate the duration of stimulus presentations, and drop NaN durations
    stimulus_presentations['duration'] = stimulus_presentations['end_time'] - stimulus_presentations['start_time']
    stimulus_presentations.dropna(subset='duration', inplace=True)
    
    # Warn if window size is too big
    if np.any(window > stimulus_presentations['duration']):
        print('Warning: window size longer than stimulus presentation')

    responses_by_unit = list()
    for iu in units.index:
        unit_spike_times = sorted_spikes[iu]

        # Determine the first and last spike time for each stimulus presentation
        start_is = np.searchsorted(unit_spike_times, stimulus_presentations['start_time'])
        end_is = np.searchsorted(unit_spike_times, stimulus_presentations['start_time'] + window)

        # Calculate the response rate for each stimulus presentation
        responses_by_unit.append((end_is - start_is) / window)

    # responses_by_unit has each row as a unit, and each column as a stimulus, flip so that rows are stimuli
    responses = np.transpose(responses_by_unit)

    # Extract the labels that match the responses from our sorted stimulus presentations table
    labels = np.array(stimulus_presentations['image_name'])
    
    return responses, labels

<div class="alert alert-block alert-success">

<h2>Exercise 2.1: Decoding for Different Brain Areas, Behavioral States, and Stimuli</h2>

<p>
In the afternoon session we looked at decoding performance, restriciting the analysis to 
<ol>
<li>one brain area ('VISp'),</li>
<li>one stimulus set ('Natural_Images_Lum_Matched_set_ophys_G_2019'), and</li>
<li>only one type of trials ('active')</li>
</ol>
It's time to generalize! Let's plot the decoding performance for each brain area recorded seperated by running/not running trials. Consider further all stimulus sets with moderate amount (10-500) of stimulus conditions.

<p>
<strong>Note:</strong>  For this exercise, there are comments with detailed prompts that act as guiderails. But feel free to try completing the task objectives using your own approach first, and consulting our prompts if you get stuck.
</p>
    
</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.a:</strong> As we did in the lecture, retrieve data of session 1065437523 from the  <code>VisualBehaviorNeuropixelsProjectCache</code>.

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.b:</strong> Obtain the <strong>annotated</strong> stimulus presentations for the session using the <code>behavior_utils</code> package.

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.c:</strong> Plot histograms of <code>mean_pupil_width</code> and <code>mean_running_speed</code>.

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.d:</strong> As we just observed, in the VB dataset, the mice were nearly always running. To consider a dataset that includes trials where mice are 'not running', let's turn to the VC dataset. Retrieve data of session 798911424 from the <code>EcephysProjectCache</code>.

</div>
<details>
    <summary>Click for <strong>Hint:</strong></summary>
    The manifest is located in the data subfolder <code>allen-brain-observatory/visual-coding-neuropixels/ecephys-cache/manifest.json</code>.
</details>

<h2 align="center"> The Visual Coding (VC) stimulus sets </h2> 
<img src="../../resources/neuropixels_stimulus_sets.png">  

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.e:</strong> Retrieve unit data, sort the units by depth, and filter for 'good' units using the same criteria as in the lecture.

</div>
<details>
    <summary>Click for <strong>Hints:</strong></summary>
    Unlike for VB (Prompt 2.1.b), here the session object has no <code>get_units</code> method, but a <code>units</code> attribute that already contains channel information, thus no merging is necessary.
</details>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.f:</strong> Get the <code>stimulus_presentations</code> table for the session and create a list of the <code>stimulus_names</code> with at least 10 but less than 500 stimulus conditions.

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.g:</strong> Calculate the average running speed for each trial and add them as new column <code>'mean_running_speed'</code> to the stimulus table.

</div>
<details>
    <summary>Click for a <strong>Hint:</strong></summary>
    For VB the <code>behavior_utils</code> package provides a convenient function to obtain an annotated stimulus table, whereas here for VC we need to do spell out the calculation steps ourselves.
</details>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.h:</strong> Plot a histogram of running speeds, indicating a threshold of 5 cm/s.

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.i:</strong> Add a new column to the stimulus table named <code>'running'</code>, with values set to <code>True</code> if the running speed is greater than 5 cm/s, otherwise <code>False</code>.

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.j:</strong> Try using the <code>make_response_array</code> function using the session's spike times and stimulus table.
<p>
Why does it fail? Modify the stimulus table as needed to ensure the function succeeds. (Alternatively, you could modify the function.)
    
</div>

<details>
    <summary>Click for a <strong>Hint:</strong></summary>
    We want to decode <code>'stimulus_condition_id'</code>. Rename (or duplicate) appropriate columns to <code>'end_time'</code> and <code>'image_name'</code> respectively.
</details>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.k:</strong> Plot the number of good units (y-axis) for each brain structure (x-axis).  

</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.l:</strong> Create a function <code>decode(area_of_interest, selection, window, n_splits)</code> that returns the accuracies of stimulus decoding in <code>area_of_interest</code> for a given <code>selection</code> of stimulus presentations.
</div>

<details>
    <summary>Click for <strong>Hints:</strong></summary>
    Revisiting the steps performed in the lecture will help to create the function that
    <ol>
    <li>selects from the <code>good_units</code> the ones in the <code>area_of_interest</code> (see Prompt 2.1.e)</li>
    <li>selects the stimulus presentations from our annotated stimulus presentations table according to the <code>selection</code> dictionary (see Prompt 2.1.f+g)</li>
    <li>creates <code>responses</code> and <code>labels</code> using the function <code>make_response_array</code> with window size <code>window</code></li>
    <li>uses <code>sklearn.model_selection.KFold</code> to split the data into "train" and "test" sets for <code>n_splits</code> iterations, and for each iteration trains a <code>sklearn.svm.SVC</code> on the training set and calculates the accuracy on the test set</li>
    <li>returns a list/array of length <code>n_splits</code> with the accuracies for each split</li>
    </ol>
    The signature of the function with default args could be 
    <p>
    <code>def decode(area_of_interest='VISp',
            selection={"stimulus_name": 'natural_scenes', "running": True},
            window=.25,
            num_splits=5):</code>
    </p>
</details>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.m:</strong> Equipped with this function, 
<p> for each <code>stimulus_name</code> from Prompt 2.1.f, (i.e. 'gabors', 'drifting_gratings', 'static_gratings', 'natural_scenes', 'drifting_gratings_contrast'),
<p> &emsp;&emsp; for both <code>"running"</code> and <code>"not_running"</code> trials,
<p> &emsp;&emsp;&emsp;&emsp; calculate and store the decoding accuracy for each recorded brain <code>structure</code>.
</div>

<details>
    <summary>Click to receive a <strong>Hint</strong> for faster processing:</summary>
    You could use <code>multiprocessing.Pool(processes).map()</code> to speed up processing by applying the <code>decode</code> function to all <code>structures</code> in parallel.
    <br>You might run out of memory if you use too many <code>processes</code>. On CodeOcean, don't use more than <code>os.environ.get("CO_CPUS")</code>
</details>

<div class="alert alert-block alert-success">

<strong>Prompt 2.1.n:</strong> For each considered <code>stimulus_name</code>, create a figure with an error bar plot of decoding accuracy (y-axis) for each brain area (x-axis) for "running" trials. Add another error bar plot for "not_running" trials using a different color in the same figure, and include a horizontal line indicating chance level performance.

</div>

<div class="alert alert-block alert-success">

<h2>Exercise 2.2: Noise Correlations and Modulation by Running</h2>

<p>
In this exercise, we will explore how running modulates neural responses and affects noise correlations. Specifically, we will:
<ol>
<li>Plot total, signal, and noise correlations</li>
<li>Plot the correlations between neural responses and running speed</li>
<li>Compare the noise correlations of all units versus highly modulated units</li>
</ol>

<p>
<strong>Note:</strong> For this exercise, there are comments with detailed prompts that act as guidelines. However, feel free to try completing the task objectives using your own approach first, and consult our prompts if you get stuck.
</p>
    
</div>

<div class="alert alert-block alert-success">

<strong>Prompt 2.2.a:</strong> Make sure you have completed the steps in Prompts 2.1.d-j.

<div class="alert alert-block alert-success">
  
<strong>Prompt 2.2.b:</strong> Consider good units in <code>area_of_interest='VISp'</code> and restrict the stimulus presentations to <code>selection={"stimulus_name": 'drifting_gratings', "running": True}</code>. Using a window size of 250ms, calculate the responses and labels using the <code>make_response_array</code> function.

<div class="alert alert-block alert-success">

<strong>Prompt 2.2.c:</strong> Calculate and plot total, signal, and mean noise correlations.


<div class="alert alert-block alert-success">
    
<strong>Prompt 2.2.d:</strong> For each stimulus condition, calculate the correlation with <code>'mean_running_speed'</code> for each unit. Plot the array (size #conditions x #units) of running speed correlations.

<div class="alert alert-block alert-success">

<strong>Prompt 2.2.e:</strong> Consider the stimulus condition with the most stimulus presentations. For this condition, plot the <code>'speed_correlations'</code>, showing a threshold of 0.75. Get the indices of the <code>'highly_modulated_units'</code> whose correlation with running speed is greater than 0.75.

<div class="alert alert-block alert-success">
    
<strong>Prompt 2.2.f:</strong> Create plots comparing the noise correlations of <code>'highly_modulated_units'</code> to all units.
    
<ol>
<li>Plot the responses (for the considered condition) of those <code>'highly_modulated_units'</code> together with the running speed (use a separate y-axis for the latter).</li>
<li>In another panel, plot the noise correlations (for the considered condition) of the <code>'highly_modulated_units'</code>.</li>
<li>In another panel, plot the histogram of the off-diagonal elements of the noise correlations (for the considered condition) of (1) all units and (2) <code>'highly_modulated_units'</code> (use density, not counts).</li>
</ol>

<div class="alert alert-block alert-success">

<strong>Prompt 2.2.g:</strong> Create synthetic data with removed noise correlations by trial-shuffling the neural data (for each condition).

<div class="alert alert-block alert-success">

<strong>Prompt 2.2.h:</strong> Decode the stimuli using a linear SVM on the true and shuffled responses. Repeatedly split into 'train' and 'test' data using <code>StratifiedKFold</code> or <code>LeaveOneOut</code> and calculate the accuracies averaged over splits.
<p> Why not just use KFold as we did in the lecture?
</div>

<details>
    <summary>Click for a <strong>Hint:</strong></summary>
    Consider the number of presentations of each stimulus condition.
</details>

<div class="alert alert-block alert-success">

<strong>Prompt 2.2.i:</strong> We considered <code>'drifting_gratings'</code>. Change to a static stimulus set and run all cells of this exercise again.